In [1]:
#from john
import numpy as np
import tensorflow as tf
import os
import time
import sys
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from datetime import datetime
import statistics as stat
import ROOT

2023-06-08 19:48:16.305724: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.3-6fa5c/x86_64-centos7-gcc11-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.3-c1d8e/x86_64-centos7-gcc11-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/jaxlib/mlir/_mlir_libs:/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/onnxruntime/capi/:/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow/contrib

Welcome to JupyROOT 6.28/00


In [2]:
# tf.keras.models.Model?

In [3]:
#mine
import uproot
import pandas as pd
import numpy as np
import boost_histogram as bh
import matplotlib.pyplot as plt
import pickle
import atlasplots as ap
%matplotlib inline

In [4]:
df_gjfull = pickle.load(open('picklefiles/df_gj_800664.pickle', 'rb'))
df_jjfull = pickle.load(open('picklefiles/df_jj_801279.pickle', 'rb'))

In [5]:
df_gjfull.index%2 == 0

array([ True, False,  True, ...,  True, False,  True])

In [6]:
len(df_gjfull.index),len(df_jjfull.index)

(2373013, 532794)

In [7]:
len(df_gjfull[df_gjfull.y_isTruthMatchedPhoton].index)

2275239

In [8]:
fulldf = pd.concat([df_gjfull,df_jjfull])
fulldf

,evtWeight,mcWeight,mcTotWeight,yWeight,y_passOQ,y_pt,y_eta,y_isTruthMatchedPhoton,y_convType,y_truth_convType,...,y_Reta,y_weta2,y_Rphi,y_wtots1,y_weta1,y_fracs1,y_deltae,y_Eratio,y_f1,HadLeakage
0,1.0,1.0,2.760393e-01,0.0,True,409.209595,1.753538,True,0,0,...,0.964968,0.009346,0.978234,2.002914,0.583963,0.207605,43.101593,0.989079,0.081320,0.002945
1,1.0,1.0,2.760393e-01,0.0,True,315.108459,0.265245,True,0,1,...,0.975056,0.008734,0.988232,1.719431,0.554444,0.203250,46.003128,0.989451,0.097205,0.007725
2,1.0,1.0,2.760393e-01,0.0,True,470.509003,1.089436,True,4,1,...,0.961069,0.010051,0.966097,2.142255,0.638442,0.308223,0.000000,0.998809,0.218940,-0.003068
3,1.0,1.0,2.760393e-01,0.0,True,317.702118,-0.267235,True,0,0,...,0.976663,0.008390,0.984343,1.783198,0.550030,0.177771,26.000320,0.939906,0.056986,0.004741
4,1.0,1.0,2.760393e-01,0.0,True,292.126465,0.317192,True,3,1,...,0.970260,0.008896,0.981166,1.699356,0.593774,0.181963,9.999687,0.995281,0.196796,0.001473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532789,1.0,1.0,2.152951e+06,0.0,True,35.467777,0.535228,False,1,1,...,0.963726,0.009265,0.853047,3.015149,0.567516,0.980156,710.852783,0.012008,0.303379,0.009098
532790,1.0,1.0,2.152951e+06,0.0,True,26.813547,-2.078789,True,0,1,...,0.933606,0.011496,0.948996,1.256975,0.600406,0.149581,129.003021,0.963583,0.250500,-0.007825
532791,1.0,1.0,2.152951e+06,0.0,True,25.979445,-0.427861,True,0,0,...,0.916155,0.010416,0.973931,2.561163,0.533667,0.189209,109.997543,0.875877,0.260899,-0.016072
532792,1.0,1.0,2.152951e+06,0.0,True,35.109184,1.003924,False,0,0,...,0.949494,0.009419,0.967284,2.458275,0.654840,0.322828,41.003250,0.967088,0.305182,-0.017902


In [9]:
fulldf.index = list(range(len(fulldf)))   #resetting indices
fulldf

,evtWeight,mcWeight,mcTotWeight,yWeight,y_passOQ,y_pt,y_eta,y_isTruthMatchedPhoton,y_convType,y_truth_convType,...,y_Reta,y_weta2,y_Rphi,y_wtots1,y_weta1,y_fracs1,y_deltae,y_Eratio,y_f1,HadLeakage
0,1.0,1.0,2.760393e-01,0.0,True,409.209595,1.753538,True,0,0,...,0.964968,0.009346,0.978234,2.002914,0.583963,0.207605,43.101593,0.989079,0.081320,0.002945
1,1.0,1.0,2.760393e-01,0.0,True,315.108459,0.265245,True,0,1,...,0.975056,0.008734,0.988232,1.719431,0.554444,0.203250,46.003128,0.989451,0.097205,0.007725
2,1.0,1.0,2.760393e-01,0.0,True,470.509003,1.089436,True,4,1,...,0.961069,0.010051,0.966097,2.142255,0.638442,0.308223,0.000000,0.998809,0.218940,-0.003068
3,1.0,1.0,2.760393e-01,0.0,True,317.702118,-0.267235,True,0,0,...,0.976663,0.008390,0.984343,1.783198,0.550030,0.177771,26.000320,0.939906,0.056986,0.004741
4,1.0,1.0,2.760393e-01,0.0,True,292.126465,0.317192,True,3,1,...,0.970260,0.008896,0.981166,1.699356,0.593774,0.181963,9.999687,0.995281,0.196796,0.001473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2905802,1.0,1.0,2.152951e+06,0.0,True,35.467777,0.535228,False,1,1,...,0.963726,0.009265,0.853047,3.015149,0.567516,0.980156,710.852783,0.012008,0.303379,0.009098
2905803,1.0,1.0,2.152951e+06,0.0,True,26.813547,-2.078789,True,0,1,...,0.933606,0.011496,0.948996,1.256975,0.600406,0.149581,129.003021,0.963583,0.250500,-0.007825
2905804,1.0,1.0,2.152951e+06,0.0,True,25.979445,-0.427861,True,0,0,...,0.916155,0.010416,0.973931,2.561163,0.533667,0.189209,109.997543,0.875877,0.260899,-0.016072
2905805,1.0,1.0,2.152951e+06,0.0,True,35.109184,1.003924,False,0,0,...,0.949494,0.009419,0.967284,2.458275,0.654840,0.322828,41.003250,0.967088,0.305182,-0.017902


In [10]:
def shuffle_and_split_data(data, test_ratio):
    '''from the textbook. splits into training and testing'''
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [11]:
# selecting just the desired variables
# fulldf_desired = 

In [12]:
#plitting into testing and training
train_set_full, test = shuffle_and_split_data(fulldf, 0.2)
train, valid = shuffle_and_split_data(train_set_full, 0.1)

In [13]:
len(train), len(valid), len(test)

(2092182, 232464, 581161)

In [14]:
#features we want.
variables = ['HadLeakage','y_Reta','y_Rphi','y_weta2','y_wtots1','y_weta1','y_fracs1', 'y_deltae','y_Eratio', 'y_f1']

In [15]:
trainfeatures = train[variables] #input to NN
trainlabels = train['y_isTruthMatchedPhoton']*1
trainweights = train['mcTotWeight']  #CHANGE THIS LATER WHEN WORKING WITH mc20

validfeatures = valid[variables] #input to NN
validlabels = valid['y_isTruthMatchedPhoton']*1
validweights = valid['mcTotWeight']  #CHANGE THIS LATER WHEN WORKING WITH mc20

testfeatures = test[variables] #input to NN
testlabels = test['y_isTruthMatchedPhoton']*1
testweights = test['mcTotWeight']  #CHANGE THIS LATER WHEN WORKING WITH mc20


In [16]:
trainlabels*1

994884     1
79097      1
1677741    1
2633153    0
2749987    0
          ..
884353     1
1212766    1
350643     1
1200496    1
1357788    1
Name: y_isTruthMatchedPhoton, Length: 2092182, dtype: int64

In [23]:
print(f'{np.sum(validlabels)/len(validlabels) * 100:.2f}% are True photons. \nThis is the accuracy a dummy prediction would give')

80.30% are True photons. 
This is the accuracy a dummy prediction would give


In [18]:
#LETS TRY THE NN FROM THE BOOK (SEQUENTIAL)
import tensorflow as tf

tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=[10]))
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

2023-06-08 19:49:47.697538: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.3-6fa5c/x86_64-centos7-gcc11-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.3-c1d8e/x86_64-centos7-gcc11-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/jaxlib/mlir/_mlir_libs:/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/onnxruntime/capi/:/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_fo

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 300)               3300      
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 33,501
Trainable params: 33,501
Non-trainable params: 0
_________________________________________________________________


In [45]:
# tf.keras.utils.plot_model(model, "my_fashion_mnist_model.png", show_shapes=True)

In [46]:
# weights, biases = model.layers[1].get_weights()
# weights

In [20]:
model.compile(loss="binary_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [21]:
# np.array(trainlabels*1)#, np.array(trainfeatures)

In [22]:
history = model.fit(np.array(trainfeatures), np.array(trainlabels*1), epochs=5,
                    validation_data=(np.array(validfeatures), np.array(validlabels*1)))

Epoch 1/5
65381/65381 [==============================] - 138s 2ms/step - loss: 38.5824 - accuracy: 0.8030 - val_loss: 0.4963 - val_accuracy: 0.8030
Epoch 2/5
65381/65381 [==============================] - 125s 2ms/step - loss: 0.4965 - accuracy: 0.8028 - val_loss: 0.4963 - val_accuracy: 0.8030
Epoch 3/5
65381/65381 [==============================] - 134s 2ms/step - loss: 0.4965 - accuracy: 0.8028 - val_loss: 0.4963 - val_accuracy: 0.8030
Epoch 4/5
65381/65381 [==============================] - 135s 2ms/step - loss: 0.4965 - accuracy: 0.8028 - val_loss: 0.4963 - val_accuracy: 0.8030
Epoch 5/5
65381/65381 [==============================] - 124s 2ms/step - loss: 0.4965 - accuracy: 0.8028 - val_loss: 0.4963 - val_accuracy: 0.8030


In [24]:
##partially from john, just the beginning of his function

def train_model(train_features,train_labels,train_weights,valid_features,valid_labels,valid_weights):
    inputs = tf.keras.layers.Input(dtype=tf.float64,shape=(train_features.shape[1],))
    Dx = inputs
    Dx = tf.keras.layers.BatchNormalization()(Dx)
    Dx = tf.keras.layers.Dense(512,input_shape = (train_features.shape[1],))(Dx)
    Dx = tf.keras.layers.LeakyReLU()(Dx)
    Dx = tf.keras.layers.Dense(512,input_shape = (train_features.shape[1],))(Dx)
    Dx = tf.keras.layers.LeakyReLU()(Dx)
    Dx = tf.keras.layers.Dense(512,input_shape = (train_features.shape[1],))(Dx)
    Dx = tf.keras.layers.Dense(1,activation="sigmoid")(Dx)
    f = tf.keras.models.Model([inputs],[Dx])

    print('Classifier Summary')
    f.summary()
    return f

In [25]:
model2 = train_model(np.array(trainfeatures), np.array(trainlabels), np.array(trainweights), np.array(validfeatures), np.array(validlabels), np.array(validweights))

Classifier Summary
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10)]              0         
                                                                 
 batch_normalization (BatchN  (None, 10)               40        
 ormalization)                                                   
                                                                 
 dense_3 (Dense)             (None, 512)               5632      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 512)               0         
                                          

In [26]:
model2.compile(loss = tf.keras.losses.binary_crossentropy, optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5),metrics=["accuracy"])

In [27]:
hist = model2.fit(x=np.array(trainfeatures),
       y=np.array(trainlabels),
       sample_weight=np.abs(np.array(trainweights)),
       batch_size=256,
       epochs=5,
       verbose=1,
       validation_data=(np.array(validfeatures),np.array(validlabels),np.abs(np.array(validweights))))


Epoch 1/5
8173/8173 [==============================] - 400s 49ms/step - loss: 85039.1016 - accuracy: 0.7522 - val_loss: 88334.7188 - val_accuracy: 0.7978
Epoch 2/5
8173/8173 [==============================] - 327s 40ms/step - loss: 81515.3359 - accuracy: 0.8116 - val_loss: 87329.2656 - val_accuracy: 0.7814
Epoch 3/5
8173/8173 [==============================] - 328s 40ms/step - loss: 80585.0625 - accuracy: 0.8350 - val_loss: 87462.7031 - val_accuracy: 0.8295
Epoch 4/5
8173/8173 [==============================] - 321s 39ms/step - loss: 80107.8281 - accuracy: 0.8425 - val_loss: 86126.9141 - val_accuracy: 0.8299
Epoch 5/5
8173/8173 [==============================] - 328s 40ms/step - loss: 79713.2109 - accuracy: 0.8522 - val_loss: 86610.7812 - val_accuracy: 0.8288


In [43]:
## try with mc20 ntuples:
df_mc20_gj = pickle.load(open('picklefiles/df_mc20_gj.pickle', 'rb'))
df_mc20_jj = pickle.load(open('picklefiles/df_mc20_jj.pickle', 'rb'))

fulldf_mc20 = pd.concat([df_mc20_gj,df_mc20_jj])
fulldf_mc20.index = list(range(len(fulldf_mc20)))
fulldf_mc20

,evtWeight,mcWeight,mcTotWeight,yWeight,y_passOQ,y_pt,y_eta,y_isTruthMatchedPhoton,y_convType,y_truth_convType,...,HadLeakage_stand,y_Reta_stand,y_Rphi_stand,y_weta2_stand,y_wtots1_stand,y_weta1_stand,y_fracs1_stand,y_deltae_stand,y_Eratio_stand,y_f1_stand
0,0.694366,1.0,1376.094295,1.012471,True,15.728769,0.401363,True,0,0,...,-0.262049,0.099062,0.453958,-0.156915,0.155808,0.063540,-0.547942,-0.340894,0.816206,-1.319150
1,0.231644,1.0,459.071480,0.944613,True,16.643381,1.317189,True,0,1,...,-0.287403,0.476325,-0.370169,-0.227834,0.155940,0.064833,-0.330285,-0.429634,0.566705,-0.156115
2,1.083721,1.0,2147.718576,0.955260,True,11.592933,2.353105,True,0,0,...,-0.413110,0.104631,0.371826,0.201361,0.155525,0.063301,-1.022912,0.144086,-0.041679,-1.817383
3,1.042794,1.0,2066.610245,0.944097,True,14.840798,1.032143,True,0,0,...,-0.367691,0.234193,0.641236,-1.370602,0.156241,0.064293,0.100570,-0.379988,-0.259924,-2.428030
4,0.997500,1.0,1976.846219,1.000000,True,9.218862,-0.988313,True,0,0,...,-0.316146,0.008778,0.565031,-0.518954,0.155634,0.063791,-0.104108,-0.402867,0.610984,-1.427562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500395,0.894000,1.0,22927.200181,1.000000,True,8.135542,0.187085,False,0,0,...,0.539033,0.031830,-0.743727,1.217401,0.157458,0.062926,-0.074428,-0.402863,-0.161259,0.766782
500396,1.069210,1.0,27420.556830,1.026242,True,10.881602,-0.072192,False,0,0,...,-0.035198,-0.039292,-2.438031,-0.831372,0.156333,0.064482,0.304487,-0.224009,-0.738630,0.855115
500397,0.947560,1.0,24300.782005,0.976738,True,11.723179,1.341223,False,0,0,...,-0.177766,-0.446540,-1.761245,1.188863,0.156863,0.066615,0.603618,-0.386219,-0.541058,0.398905
500398,1.028350,1.0,26372.684195,1.041827,True,14.162736,1.779504,False,0,1,...,2.207922,-0.255052,-0.510255,0.028431,0.156257,0.066366,0.447285,-0.335196,-0.141033,0.051943


In [29]:
train_set_full2, test2 = shuffle_and_split_data(fulldf_mc20, 0.2)
train2, valid2 = shuffle_and_split_data(train_set_full2, 0.1)  #not doing test rn cause too small

train2features = train2[variables] #input to NN
train2labels = train2['y_isTruthMatchedPhoton']
train2weights = train2['goodWeight']

valid2features = valid2[variables] #input to NN
valid2labels = valid2['y_isTruthMatchedPhoton']
valid2weights = valid2['goodWeight']

test2features = test2[variables] #input to NN
test2labels = test2['y_isTruthMatchedPhoton']*1
test2weights = test2['goodWeight']


In [36]:
print(f'{np.sum(valid2labels)/len(valid2labels) * 100:.2f}% are True photons. \nThis is the accuracy a dummy prediction would give')

48.50% are True photons. 
This is the accuracy a dummy prediction would give


In [39]:
modelmc20 = tf.keras.Sequential()
modelmc20.add(tf.keras.layers.InputLayer(input_shape=[10]))
modelmc20.add(tf.keras.layers.Dense(300, activation="relu"))
modelmc20.add(tf.keras.layers.Dense(100, activation="relu"))
modelmc20.add(tf.keras.layers.Dense(1, activation="sigmoid"))


modelmc20.compile(loss="binary_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [40]:
history_mc20 = modelmc20.fit(np.array(train2features), np.array(train2labels*1), sample_weight=np.abs(np.array(train2weights)), epochs=5,
                    validation_data=(np.array(valid2features), np.array(valid2labels*1), np.abs(np.array(valid2weights)) ) )

Epoch 1/5
11259/11259 [==============================] - 34s 3ms/step - loss: nan - accuracy: 0.5190 - val_loss: nan - val_accuracy: 0.5150
Epoch 2/5
 9507/11259 [========================>.....] - ETA: 3s - loss: nan - accuracy: 0.5187

KeyboardInterrupt: 

In [ ]:
###why is the loss nan for this ^^ whatever its using as the loss function isnt working. maybe has to do with weights

In [41]:
model2mc20 = train_model(np.array(train2features), np.array(train2labels), np.array(train2weights), np.array(valid2features), np.array(valid2labels), np.array(valid2weights))

model2mc20.compile(loss = tf.keras.losses.binary_crossentropy, optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5),metrics=["accuracy"])

Classifier Summary
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 10)]              0         
                                                                 
 batch_normalization_1 (Batc  (None, 10)               40        
 hNormalization)                                                 
                                                                 
 dense_13 (Dense)            (None, 512)               5632      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_14 (Dense)            (None, 512)               262656    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                        

In [42]:
hist_mc20 = model2mc20.fit(x=np.array(train2features),
       y=np.array(train2labels),
       sample_weight=np.abs(np.array(train2weights)),
       batch_size=256,
       epochs=25,
       verbose=1,
       validation_data=(np.array(valid2features),np.array(valid2labels),np.abs(np.array(valid2weights))))

Epoch 1/25
1408/1408 [==============================] - 71s 50ms/step - loss: 2108.2300 - accuracy: 0.6382 - val_loss: 2118.8369 - val_accuracy: 0.5267
Epoch 2/25
1408/1408 [==============================] - 56s 40ms/step - loss: 1853.0654 - accuracy: 0.6750 - val_loss: 1876.7306 - val_accuracy: 0.5949
Epoch 3/25
1408/1408 [==============================] - 56s 40ms/step - loss: 1831.5443 - accuracy: 0.6784 - val_loss: 1855.8752 - val_accuracy: 0.6141
Epoch 4/25
1408/1408 [==============================] - 54s 39ms/step - loss: 1820.7915 - accuracy: 0.6801 - val_loss: 1834.9055 - val_accuracy: 0.6128
Epoch 5/25
1408/1408 [==============================] - 55s 39ms/step - loss: 1813.5723 - accuracy: 0.6808 - val_loss: 1821.1704 - val_accuracy: 0.6407
Epoch 6/25
1408/1408 [==============================] - 56s 40ms/step - loss: 1808.1006 - accuracy: 0.6826 - val_loss: 1901.7656 - val_accuracy: 0.5783
Epoch 7/25
1408/1408 [==============================] - 56s 40ms/step - loss: 1802.3380 

In [106]:
#now its overfitting because of insufficient data. Hmmm what to do. or maybe not, might just need to let it train longer.
#make plots of performance next week and see

In [ ]:
#now with standaridzed variables

In [44]:
standvars = [var+'_stand' for var in variables]
variables, standvars

(['HadLeakage',
  'y_Reta',
  'y_Rphi',
  'y_weta2',
  'y_wtots1',
  'y_weta1',
  'y_fracs1',
  'y_deltae',
  'y_Eratio',
  'y_f1'],
 ['HadLeakage_stand',
  'y_Reta_stand',
  'y_Rphi_stand',
  'y_weta2_stand',
  'y_wtots1_stand',
  'y_weta1_stand',
  'y_fracs1_stand',
  'y_deltae_stand',
  'y_Eratio_stand',
  'y_f1_stand'])

In [46]:
train_set_full3, test3 = shuffle_and_split_data(fulldf_mc20, 0.2)
train3, valid3 = shuffle_and_split_data(train_set_full3, 0.1)  

train3features = train3[standvars] #input to NN
train3labels = train3['y_isTruthMatchedPhoton']
train3weights = train3['goodWeight']

valid3features = valid3[standvars] #input to NN
valid3labels = valid3['y_isTruthMatchedPhoton']
valid3weights = valid3['goodWeight']

test3features = test3[standvars] #input to NN
test3labels = test3['y_isTruthMatchedPhoton']*1
test3weights = test3['goodWeight']


In [47]:
tf.random.set_seed(52)
modelstand = tf.keras.Sequential()
modelstand.add(tf.keras.layers.InputLayer(input_shape=[10]))
modelstand.add(tf.keras.layers.Dense(300, activation="relu"))
modelstand.add(tf.keras.layers.Dense(100, activation="relu"))
modelstand.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [48]:
modelstand.compile(loss="binary_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [50]:
history_stand = modelstand.fit(np.array(train3features), np.array(train3labels*1), epochs=5,
                    validation_data=(np.array(valid3features), np.array(valid3labels*1)) )

Epoch 1/5
11259/11259 [==============================] - 29s 3ms/step - loss: nan - accuracy: 0.5188 - val_loss: nan - val_accuracy: 0.5187
Epoch 2/5
11259/11259 [==============================] - 25s 2ms/step - loss: nan - accuracy: 0.5188 - val_loss: nan - val_accuracy: 0.5187
Epoch 3/5
11259/11259 [==============================] - 29s 3ms/step - loss: nan - accuracy: 0.5188 - val_loss: nan - val_accuracy: 0.5187
Epoch 4/5
11259/11259 [==============================] - 31s 3ms/step - loss: nan - accuracy: 0.5188 - val_loss: nan - val_accuracy: 0.5187
Epoch 5/5
11259/11259 [==============================] - 33s 3ms/step - loss: nan - accuracy: 0.5188 - val_loss: nan - val_accuracy: 0.5187


In [51]:
## okay yeah its got to be some problem with the weights. hm or not

In [52]:
model2stand = train_model(np.array(train3features), np.array(train3labels), np.array(train3weights), np.array(valid3features), np.array(valid3labels), np.array(valid3weights))

model2stand.compile(loss = tf.keras.losses.binary_crossentropy, optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5),metrics=["accuracy"])

Classifier Summary
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 10)]              0         
                                                                 
 batch_normalization_2 (Batc  (None, 10)               40        
 hNormalization)                                                 
                                                                 
 dense_20 (Dense)            (None, 512)               5632      
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 512)               262656    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 512)               0         
                                        

In [53]:
hist_stand = model2stand.fit(x=np.array(train3features),
       y=np.array(train3labels),
       sample_weight=np.abs(np.array(train3weights)),
       batch_size=256,
       epochs=15,
       verbose=1,
       validation_data=(np.array(valid3features),np.array(valid3labels),np.abs(np.array(valid3weights))))

Epoch 1/15
1408/1408 [==============================] - 67s 47ms/step - loss: 2080.1045 - accuracy: 0.6363 - val_loss: 1952.0164 - val_accuracy: 0.5464
Epoch 2/15
1408/1408 [==============================] - 55s 39ms/step - loss: 1837.5813 - accuracy: 0.6796 - val_loss: 2233.0793 - val_accuracy: 0.5205
Epoch 3/15
1408/1408 [==============================] - 56s 40ms/step - loss: 1819.0461 - accuracy: 0.6815 - val_loss: 1950.4596 - val_accuracy: 0.5406
Epoch 4/15
1408/1408 [==============================] - 55s 39ms/step - loss: 1809.3525 - accuracy: 0.6833 - val_loss: 1957.4504 - val_accuracy: 0.5526
Epoch 5/15
1408/1408 [==============================] - 56s 40ms/step - loss: 1799.5564 - accuracy: 0.6852 - val_loss: 1794.8757 - val_accuracy: 0.6261
Epoch 6/15
1408/1408 [==============================] - 55s 39ms/step - loss: 1797.3027 - accuracy: 0.6851 - val_loss: 1861.1556 - val_accuracy: 0.5966
Epoch 7/15
1408/1408 [==============================] - 56s 40ms/step - loss: 1791.1000 

In [ ]:
## the standardized variables have helped the accuracy a bit, and seem to allow the NN to train faster. If i can get the
#unskewing to work on the skewed variables too that would be ideal. Maybe try scaling to (0,1] and then sqrt().